# Model
This example is from Galvanin et al. 2007, *Model-Based Design of Parallel Experiments*.

$$
\begin{align}
    \frac{dx_1}{dt} &= (r - u_1 - \theta_4)x_1 \\
    \frac{dx_2}{dt} &= - \frac{rx_1}{\theta_3} + u_1(u_2 - x_2)\\
    r &= \frac{\theta_1 x_2}{\theta_2 + x_2}
\end{align}
$$

Where,  <br>
$x_1$ = biomass concentration ($g/L$). <br>
$x_2$ = substrate concentration ($g/L$).  <br>
$u_1$ = dilution factor ($h^{-1}$).  <br>
$u_2$ = substrate concentration in the feed ($g/L$).  <br>

**Experimental conditions** <br>
$x_1^0$ = initial biomass concentration ($1-10\; g/L$). <br>
$u_1 (t)$ = dilution factor ($0.05 - 0.20\; h^{-1}$).  <br>
$u_2 (t)$ = substrate concentration in the feed ($5-35\; g/L$).  <br>


**Measurements** <br>
$x_1 (t)$ = biomass concentration ($g/L$) <br>
$x_2 (t)$ = substrate concentration ($g/L$)  <br>

**Parameters** <br>
$\theta_1 (h^{-1}), \theta_2 (g/L), \theta_3 (\text{dimensionless}), \text{ \& } \theta_4 (h^{-1})$ <br>

**Other information** <br>
$x_2^0$ = initial substrate concentration ($0\; g/L$), cannot be manipulated. <br>
Total duration of the experiment, $\tau$ = $40\; h$ (fixed). <br>
Each experimental run involves 5 sampling times. <br>
Inputs $\mathbf{u}(t)$ can be manipulated and represented as a piece-wise constant profile over 5 switching intervals. <br>

Although in the paper the did the following: <br>
* The sampling times and the control variables switching times can be different. <br>
* The elapsed time between any two sampling points is allowed to be between 0.1 and 20 h, 
and the duration of each control interval is allowed to be between 0.2 and 20 h. <br>

In this example, the times is kept fixed. <br>
# Simulation with True parameters

In [1]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar, Simulator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar
from pyomo.dae.simulator import Simulator
import numpy as np
import pandas as pd

# True parameter values for the mechanistic model
theta_true = [0.310, 0.180, 0.550, 0.050]


def create_biomass_model(
    time_horizon=40, theta_initial=None
):
    """Creates a Pyomo model for biomass growth."""
    m = pyo.ConcreteModel()

    # Times Set
    m.t = ContinuousSet(bounds=(0, time_horizon))

    # Parameters (fixed during simulation)
    m.theta1 = pyo.Var(initialize=theta_initial[0] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta2 = pyo.Var(initialize=theta_initial[1] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta3 = pyo.Var(initialize=theta_initial[2] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta4 = pyo.Var(initialize=theta_initial[3] if theta_initial else 1, domain=pyo.PositiveReals)

    # Time-varying inputs
    m.u1 = pyo.Var(m.t)
    m.u2 = pyo.Var(m.t)

    # State Variables
    m.x1 = pyo.Var(m.t, domain=pyo.NonNegativeReals)
    m.x2 = pyo.Var(m.t, domain=pyo.NonNegativeReals)

    # Derivatives
    m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
    m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

    # ODEs - with r inlined to avoid Expression evaluation issues
    def _dx1dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx1dt[t] == (r - m.u1[t] - m.theta4) * m.x1[t]

    m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)

    def _dx2dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx2dt[t] == -r * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])

    m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)

    return m


def generate_synthetic_data(initial_x1, u1_profile, u2_profile, noise_matrix="A"):
    """Generates synthetic data for biomass growth model."""
    model = create_biomass_model(time_horizon=40)

    # Set true parameter values
    model.theta1.fix(theta_true[0])
    model.theta2.fix(theta_true[1])
    model.theta3.fix(theta_true[2])
    model.theta4.fix(theta_true[3])

    # Set initial condition
    model.x1[0].fix(initial_x1)
    model.x2[0].fix(0.0)

    # Initialize Simulator
    sim = Simulator(model, package='scipy')

    # Create Suffix for varying inputs
    varying_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
    varying_inputs[model.u1] = u1_profile
    varying_inputs[model.u2] = u2_profile

    # Simulate - returns (time_array, profiles_array)
    tsim = np.linspace(0, 40, 401)
    tsim_result, profiles = sim.simulate(
        numpoints=len(tsim),
        integrator='vode',
        varying_inputs=varying_inputs,
    )

    # Create DataFrame from time and profiles
    df = pd.DataFrame({'t': tsim_result, 'x1': profiles[:, 0], 'x2': profiles[:, 1]})

    # Select Measurement Points
    sample_times = [1, 5, 15, 20, 30]
    df_measured = df[df["t"].isin(sample_times)].copy()

    # Add Measurement Noise
    std_devs = {}
    if noise_matrix == "A":
        std_devs = {"x1": np.sqrt(0.01), "x2": np.sqrt(0.05)}
    elif noise_matrix == "B":
        std_devs = {"x1": np.sqrt(0.1), "x2": np.sqrt(0.5)}
    else:
        raise ValueError("Invalid noise matrix type. Choose 'A' or 'B'.")

    np.random.seed(42)
    df_measured["x1_noisy"] = df_measured["x1"] + np.random.normal(
        0, std_devs["x1"], size=len(df_measured)
    )
    df_measured["x2_noisy"] = df_measured["x2"] + np.random.normal(
        0, std_devs["x2"], size=len(df_measured)
    )

    return df, df_measured


# Example usage
controls_dict = {0: (0.15, 30.0), 10: (0.20, 10.0), 25: (0.05, 20.0)}
u1_profile = {t: vals[0] for t, vals in controls_dict.items()}
u2_profile = {t: vals[1] for t, vals in controls_dict.items()}
true_data, measured_data = generate_synthetic_data(
    initial_x1=5, u1_profile=u1_profile, u2_profile=u2_profile
)

In [3]:
true_data

,t,x1,x2
0,0.0,5.000000,0.000000
1,0.1,4.969858,0.321184
2,0.2,4.979694,0.565755
3,0.3,5.001896,0.783783
4,0.4,5.030735,0.985666
...,...,...,...
396,39.6,5.259586,0.091078
397,39.7,5.261761,0.091020
398,39.8,5.263913,0.090964
399,39.9,5.266044,0.090908


In [4]:
measured_data

,t,x1,x2,x1_noisy,x2_noisy
10,1.0,5.265617,2.000276,5.315288,1.947921
50,5.0,7.694514,4.118824,7.680688,4.471947
150,15.0,6.391754,0.206251,6.456523,0.377854
200,20.0,4.804952,0.406633,4.957255,0.301655
300,30.0,4.905243,0.101489,4.881827,0.222809


# Create the Experiment Class

In [5]:
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.parmest import parmest
from pyomo.contrib.doe import DesignOfExperiments


In [20]:
class BiomassExperiment(Experiment):

    def __init__(
        self,
        data_df,
        theta_initial,
        u1_profile,
        u2_profile,
        x1_initial=5.0,
        x2_initial=0.0,
        time_horizon=40,
        measurment_error_matrix="A",
        simulation_initialization=False,
        scheme='LAGRANGE-RADAU',
        nfe=10,
        ncp=3,
        optimize_sampling_times=False,
        optimize_control_times=False,
    ):
        """Biomass Experiment class for parameter estimation and design of experiments.

        Parameters
        ----------
        data_df : pandas.DataFrame
            The measured data for the experiment.
        theta_initial : list
            Initial guess for the parameters [theta1, theta2, theta3, theta4].
        u1_profile : dict
            The profile for u1 (dilution factor).
        u2_profile : dict
            The profile for u2 (substrate concentration in feed).
        x1_initial : float
            Initial biomass concentration.
        x2_initial : float
            Initial substrate concentration.
        time_horizon : float
            The time horizon for the experiment.
        measurment_error_matrix : str
            Type of measurement error matrix ("A" or "B").
        nfe : int
            Number of finite elements for discretization.
        ncp : int
            Number of collocation points per finite element.
        optimize_sampling_times : bool
            Whether to optimize sampling times (t_sample) during DOE.
            If False, sampling times are fixed to the values in data_df.
        optimize_control_times : bool
            Whether to optimize control switching times (t_control) during DOE.
            If False, control times are fixed to the times in u1_profile/u2_profile.
        """
        self.data = data_df
        self.theta_initial = theta_initial
        self.u1_profile = u1_profile
        self.u2_profile = u2_profile
        self.x1_initial = x1_initial
        self.x2_initial = x2_initial
        self.time_horizon = time_horizon

        if measurment_error_matrix == "A":
            self.measurement_std_devs = {"x1": np.sqrt(0.01), "x2": np.sqrt(0.05)}
        elif measurment_error_matrix == "B":
            self.measurement_std_devs = {"x1": np.sqrt(0.1), "x2": np.sqrt(0.5)}
        else:
            raise ValueError("Invalid noise matrix type. Choose 'A' or 'B'.")

        self.simulation_initialization = simulation_initialization
        self.scheme = scheme
        self.nfe = nfe
        self.ncp = ncp
        self.model = None
        
        # Flags to enable optimization of sampling and control times
        self.optimize_sampling_times = optimize_sampling_times
        self.optimize_control_times = optimize_control_times

    def create_model(self):
        """Creates a Pyomo model for biomass growth."""
        m = self.model = pyo.ConcreteModel()

        # Times Set
        m.t = ContinuousSet(bounds=(0, self.time_horizon))

        # Parameters (fixed during simulation)
        m.theta1 = pyo.Var(
            initialize=self.theta_initial[0] if self.theta_initial else 1,
            bounds=(1e-6, 1),
        )
        m.theta2 = pyo.Var(
            initialize=self.theta_initial[1] if self.theta_initial else 1,
            bounds=(1e-6, 1),
        )
        m.theta3 = pyo.Var(
            initialize=self.theta_initial[2] if self.theta_initial else 1,
            bounds=(1e-6, 1),
        )
        m.theta4 = pyo.Var(
            initialize=self.theta_initial[3] if self.theta_initial else 1,
            bounds=(1e-6, 1),
        )

        # Time-varying inputs
        m.u1 = pyo.Var(m.t, bounds=(0.05, 0.20))
        m.u2 = pyo.Var(m.t, bounds=(5.0, 35.0))

        # State Variables
        m.x1 = pyo.Var(m.t, domain=pyo.NonNegativeReals)
        m.x2 = pyo.Var(m.t, domain=pyo.NonNegativeReals)

        # Derivatives
        m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
        m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

        # ===== NEW: Decision variables for optimizable sampling and control times =====
        # These variables represent time points that can be optimized by the DOE algorithm
        # to maximize information content (e.g., via D-optimal or A-optimal criteria).
        
        # Control switching times (4 switches define 5 control intervals)
        # Note: t=0 is the initial condition (not a switch), switches occur at t > 0
        # The control profile is piecewise constant between these switching times
        # Example: if t_control = [10, 20, 30, 40], we have 5 intervals:
        #          [0,10), [10,20), [20,30), [30,40), [40,end]
        # Indexed 1-4 (not 0-3) to match Pyomo conventions
        m.t_control = pyo.Var(
            range(1, 5), 
            bounds=(0, self.time_horizon), 
            initialize={1: 10, 2: 20, 3: 30, 4: 40}  # Use dict for 1-indexed variables
        )
        
        # Sampling times (5 measurements)
        # These are the time points where measurements of x1 and x2 are taken
        # The DOE algorithm can optimize these times to maximize parameter identifiability
        # Indexed 1-5 (not 0-4) to match Pyomo conventions
        m.t_sample = pyo.Var(
            range(1, 6), 
            bounds=(0, self.time_horizon), 
            initialize={1: 1, 2: 5, 3: 15, 4: 20, 5: 30}  # Use dict for 1-indexed variables
        )

        # ODEs - with r inlined to avoid Expression evaluation issues
        def _dx1dt_rule(m, t):
            r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
            return m.dx1dt[t] == (r - m.u1[t] - m.theta4) * m.x1[t]

        m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)

        def _dx2dt_rule(m, t):
            r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
            return m.dx2dt[t] == -r * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])

        m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)

        return m        

    def _apply_inputs_to_model(self, model):
        """
        Helper function to interpolate and fix inputs u1/u2 at all discretized time points.
        Uses zero-order hold (piecewise constant) interpolation.
        """
        # Sort keys for zero-order hold logic
        u1_times = sorted(self.u1_profile.keys())
        u2_times = sorted(self.u2_profile.keys())

        def get_val(t, times, profile):
            """Get the active value for time t using zero-order hold."""
            active_t = 0
            for ct in times:
                if ct <= t:
                    active_t = ct
                else:
                    break
            return profile[active_t]

        # Iterate over all time points in discretized model and fix input values
        for t in model.t:
            model.u1[t].fix(get_val(t, u1_times, self.u1_profile))
            model.u2[t].fix(get_val(t, u2_times, self.u2_profile))

    def finalize_model(self):
        m = self.model
        # Fix the parameter values (they are estimated, not decision variables)
        m.theta1.fix()
        m.theta2.fix()
        m.theta3.fix()
        m.theta4.fix()

        ### ===== Handle sampling and control times based on optimization flags =====
        
        # --- Sampling Times Logic ---
        if not self.optimize_sampling_times:
            # Case 1: Fixed sampling times (for parameter estimation)
            # Fix t_sample variables to the times specified in the data
            if self.data is not None:
                sample_times = sorted(self.data["t"].unique())
                for i in range(1, 6):  # We have 5 sampling times (indexed 1-5)
                    if i - 1 < len(sample_times):
                        m.t_sample[i].fix(sample_times[i - 1])
        else:
            # Case 2: Optimizable sampling times (for DOE)
            # Add ordering constraints: t_sample[i] + 0.1 <= t_sample[i+1]
            # This ensures: (1) monotonic ordering, (2) minimum 0.1h spacing
            for i in range(1, 5):
                m.add_component(
                    f'sampling_order_{i}',
                    pyo.Constraint(expr=m.t_sample[i] + 0.1 <= m.t_sample[i + 1])
                )
            # Ensure first sample is after t=0.1 and last is before/at horizon
            m.add_component('sampling_bounds_lower', pyo.Constraint(expr=m.t_sample[1] >= 0.1))
            m.add_component('sampling_bounds_upper', pyo.Constraint(expr=m.t_sample[5] <= self.time_horizon))

        # --- Control Times Logic ---
        if not self.optimize_control_times:
            # Case 1: Fixed control times (for parameter estimation and fixed design DOE)
            # Extract switch times from the control profiles (excluding t=0)
            all_control_times = sorted(set(list(self.u1_profile.keys()) + list(self.u2_profile.keys())))
            control_switch_times = [t for t in all_control_times if t > 0]  # Exclude t=0 (initial condition)
            
            # Fix t_control variables to these times
            for i in range(1, 5):  # We have 4 control switches (indexed 1-4)
                if i - 1 < len(control_switch_times):
                    m.t_control[i].fix(control_switch_times[i - 1])
                else:
                    # Fallback: if fewer than 4 switches provided, use default spacing
                    m.t_control[i].fix(i * 10)
        else:
            # Case 2: Optimizable control times (for advanced DOE)
            # Add ordering constraints: t_control[i] + 0.2 <= t_control[i+1]
            # This ensures: (1) monotonic ordering, (2) minimum 0.2h spacing per paper specs
            for i in range(1, 4):
                m.add_component(
                    f'control_order_{i}',
                    pyo.Constraint(expr=m.t_control[i] + 0.2 <= m.t_control[i + 1])
                )
            # Ensure first control time is after t=0.2 and last is at/before horizon
            m.add_component('control_bounds_lower', pyo.Constraint(expr=m.t_control[1] >= 0.2))
            m.add_component('control_bounds_upper', pyo.Constraint(expr=m.t_control[4] <= self.time_horizon))

        ### ===== Add time points to the continuous set before discretization =====
        # This ensures that the discretization grid includes these critical points
        
        # Add Sampling times to the continuous set
        for i in range(1, 6):
            t_val = pyo.value(m.t_sample[i])
            if t_val is not None and t_val not in m.t:
                m.t.add(t_val)
        
        # Add control switch times to the continuous set
        for i in range(1, 5):
            t_val = pyo.value(m.t_control[i])
            if t_val is not None and t_val not in m.t and t_val <= m.t.last():
                m.t.add(t_val)

        # Fix initial conditions
        m.x1[0].fix(self.x1_initial)
        m.x2[0].fix(self.x2_initial)

        # Simulation initialization (optional but recommended for good initial guesses)
        if self.simulation_initialization:
            sim = Simulator(m, package='scipy')

            # Create varying inputs Suffix using the profiles we already have
            varying_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
            varying_inputs[m.u1] = self.u1_profile
            varying_inputs[m.u2] = self.u2_profile
            # Simulate on a dense grid for smooth interpolation
            # (We use 101 points to capture the curvature of Monod kinetics)
            tsim = np.linspace(0, self.time_horizon, 101)

            # Simulate the ODE system to get state trajectories
            tsim_res, profiles = sim.simulate(
                            numpoints=len(tsim),
                            integrator='vode',
                            varying_inputs=varying_inputs
                        )
            # Store trajectories (Assumes declaration order: x1 is index 0, x2 is index 1)
            x1_guess = profiles[:, 0]
            x2_guess = profiles[:, 1]            

        ### Discretize the model using collocation or finite difference
        if self.scheme in ('LAGRANGE-RADAU', 'LAGRANGE-LEGENDRE'):
            discretizer = pyo.TransformationFactory('dae.collocation')
            discretizer.apply_to(m, nfe=self.nfe, ncp=self.ncp, scheme=self.scheme)
        elif self.scheme in ("BACKWARD", "FORWARD", "CENTRAL"):
            discretizer = pyo.TransformationFactory('dae.finite_difference')
            discretizer.apply_to(m, nfe=self.nfe, scheme=self.scheme)
        else:
            raise ValueError("Invalid discretization scheme. Choose 'LAGRANGE-RADAU' or 'LAGRANGE-LEGENDRE'.")

        ### Fix inputs at all discretized time points
        self._apply_inputs_to_model(m)

        # Initialize the state variables: simulation-based (better) or constant (simpler)
        if not self.simulation_initialization:
            for t in m.t:
                m.x1[t].set_value(5.0)  # Initial guess
                m.x2[t].set_value(0.0)  # Initial guess
        else:
            # Use simulation results interpolated to discretized time points
            for t in m.t:
                m.x1[t].set_value(np.interp(t, tsim_res, x1_guess))
                m.x2[t].set_value(np.interp(t, tsim_res, x2_guess))

        return m

    def label_experiment(self):
        """
        Label the model components for parmest and DOE.
        This tells the framework which variables are:
        - experiment_outputs: measurements (with data values)
        - experiment_inputs: design variables to optimize
        - unknown_parameters: parameters to estimate
        - measurement_error: standard deviations for each measurement
        """
        m = self.model

        # ===== Set measurement variable labels (what we observe) =====
        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        
        # Use optimized sampling times if available, otherwise use fixed data times
        if self.optimize_sampling_times:
            # When optimizing sampling times, measurements occur at m.t_sample[i]
            for i in range(1, 6):
                t_sample = pyo.value(m.t_sample[i])
                # Get corresponding measurement values from data (by index)
                m.experiment_outputs[m.x1[t_sample]] = self.data["x1_noisy"].iloc[i - 1]
                m.experiment_outputs[m.x2[t_sample]] = self.data["x2_noisy"].iloc[i - 1]
        else:
            # Fixed sampling times: use the times from data directly
            m.experiment_outputs.update((m.x1[t], x1_val) for t, x1_val in zip(self.data["t"], self.data["x1_noisy"]))
            m.experiment_outputs.update((m.x2[t], x2_val) for t, x2_val in zip(self.data["t"], self.data["x2_noisy"]))

        # ===== Set design variables / inputs labels (what the experimenter can control) =====
        m.experiment_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        
        # Control input values at switch times
        if self.optimize_control_times:
            # When optimizing control times, inputs change at m.t_control[i]
            for i in range(1, 5):
                t_control = pyo.value(m.t_control[i])
                m.experiment_inputs[m.u1[t_control]] = pyo.value(m.u1[t_control])
                m.experiment_inputs[m.u2[t_control]] = pyo.value(m.u2[t_control])
        else:
            # Fixed control times: use the times from u1_profile/u2_profile
            m.experiment_inputs.update((m.u1[t], u1_val) for t, u1_val in self.u1_profile.items())
            m.experiment_inputs.update((m.u2[t], u2_val) for t, u2_val in self.u2_profile.items())
        
        # Initial biomass concentration (can be varied in some experiments)
        m.experiment_inputs[m.x1[m.t.first()]] = None
        
        # ===== Add time variables to experiment inputs if optimizing =====
        # This allows the DOE algorithm to optimize sampling and/or control times
        
        # Add sampling times to experiment inputs if optimizing
        if self.optimize_sampling_times:
            m.experiment_inputs.update((m.t_sample[i], pyo.value(m.t_sample[i])) for i in range(1, 6))
        
        # Add control switching times to experiment inputs if optimizing
        if self.optimize_control_times:
            m.experiment_inputs.update((m.t_control[i], pyo.value(m.t_control[i])) for i in range(1, 5))

        # ===== Set Unknown Parameters labels (what we want to estimate) =====
        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update((k, pyo.value(k)) for k in [m.theta1, m.theta2, m.theta3, m.theta4])

        # ===== Set measurement standard deviations (for weighted least squares and FIM calculation) =====
        m.measurement_error = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        
        # Use optimized sampling times if available, otherwise use fixed data times
        if self.optimize_sampling_times:
            # When optimizing sampling times, errors are associated with m.t_sample[i]
            for i in range(1, 6):
                t_sample = pyo.value(m.t_sample[i])
                m.measurement_error[m.x1[t_sample]] = self.measurement_std_devs["x1"]
                m.measurement_error[m.x2[t_sample]] = self.measurement_std_devs["x2"]
        else:
            # Fixed sampling times: use the times from data directly
            m.measurement_error.update((m.x1[t], self.measurement_std_devs["x1"]) for t in self.data["t"])
            m.measurement_error.update((m.x2[t], self.measurement_std_devs["x2"]) for t in self.data["t"])

        return m

    def get_labeled_model(self):
        if self.model is None:
            self.create_model()
            self.finalize_model()
            self.label_experiment()
        return self.model


## Parmest

In [21]:
# Example usage of BiomassExperiment
theta_initial_more_accurate = [0.357, 0.153, 0.633, 0.043]
theta_initial_less_accurate = [0.527, 0.054, 0.935, 0.015]
controls_dict = {0: (0.15, 30.0), 10: (0.20, 10.0), 25: (0.05, 10.0)}
u1_profile = {t: vals[0] for t, vals in controls_dict.items()}
u2_profile = {t: vals[1] for t, vals in controls_dict.items()}

experiment = BiomassExperiment(
    data_df=measured_data,
    theta_initial=theta_initial_more_accurate,
    u1_profile=u1_profile,
    u2_profile=u2_profile,
    x1_initial=5.0,
    x2_initial=0.0,
    time_horizon=40,
    measurment_error_matrix="A",
    simulation_initialization=True,
    scheme="LAGRANGE-RADAU",
    nfe=20,
    ncp=3,
)

pest = parmest.Estimator([experiment], obj_function='SSE', tee=True)

obj, theta = pest.theta_est()
# model = experiment.get_labeled_model()

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [16]:
obj

0.3471394226633355

In [17]:
theta

theta1    0.253645
theta2    0.249814
theta3    0.439945
theta4    0.000593
dtype: float64

## DOE
### Troubleshooting DOE Optimization

When optimizing both sampling and control times simultaneously, the problem can become very challenging numerically. Here's a recommended progression:

**Step 1**: Optimize only sampling times (control times fixed)
- This is simpler and tests if time optimization works at all

**Step 2**: Optimize only control times (sampling times fixed)  
- Tests if control time optimization works independently

**Step 3**: Optimize both simultaneously (after Steps 1 & 2 succeed)
- Use solutions from Steps 1 & 2 as better initialization
- May need to adjust solver options or problem formulation

**Common issues with time optimization:**
- Poor initialization → use results from simpler cases
- Conflicting constraints → check spacing and bounds are compatible  
- Scaling issues → time variables [0, 40] vs control values [0.05, 35]
- Too many design variables → start with coarser discretization (smaller nfe)

In [23]:
# DOE with optimizable times
# Start with simpler case: optimize only sampling times first
theta_initial = [float(t) for t in theta.to_numpy()]  # Use estimated parameters from previous step
experiment = BiomassExperiment(
    data_df=measured_data,
    theta_initial=theta_initial,  # Use estimated parameters from previous step
    u1_profile=u1_profile,
    u2_profile=u2_profile,
    x1_initial=5.0,
    x2_initial=0.0,
    time_horizon=40,
    measurment_error_matrix="A",
    simulation_initialization=True,
    scheme="LAGRANGE-RADAU",
    nfe=20,
    ncp=3,
    optimize_sampling_times=True,   # Start with only sampling times
    optimize_control_times=False,    # Keep control times fixed initially
)
doe_obj = DesignOfExperiments(experiment=experiment, step=0.01, tee=True)
doe_obj.run_doe()

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

In [24]:
doe_obj.results

{'Solver Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found',
 'FIM': [[np.float64(959239.5859953763),
   np.float64(-36976.903406923986),
   np.float64(-46258.6300588294),
   np.float64(-916730.8875117518)],
  [np.float64(-36976.903406923986),
   np.float64(1611.330648551933),
   np.float64(929.370795741231),
   np.float64(37235.41482497941)],
  [np.float64(-46258.6300588294),
   np.float64(929.370795741231),
   np.float64(29772.845738832795),
   np.float64(-28791.499274960486)],
  [np.float64(-916730.8875117518),
   np.float64(37235.41482497941),
   np.float64(-28791.499274960486),
   np.float64(1091074.5004993696)]],
 'Sensitivity Matrix': [[np.float64(1.2253211224014127),
   np.float64(-0.1884771219574481),
   np.float64(0.016092831134599195),
   np.float64(-1.5023877223391235)],
  [np.float64(-2.7858824214809497),
   np.float64(0.4285575770053972),
   np.float64(1.